
# <center><font color= #F08080 > <b>CU55_Modelo agregado de estimación del gasto medio por turista</font></center>

#  <font color='green'>IV. Model development</font>

En este anexo se incluye el código utilizado durante el desarrollo 
de los modelos incluidos en el caso de uso.

## Modelo de Simulación



In [1]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "es_ES.UTF-8/es_ES.UTF-8/es_ES.UTF-8/C/es_ES.UTF-8/C"

In [2]:


## simulación turistas

library(readr)
library(dplyr)
library(tidyr)
library(purrr)
library(tibble)
library(stringr)

gasto_municipio <- read_csv("cu_55_step_01_input/CU_55_05_02_gasto_municipio.csv")

dm <- gasto_municipio |> 
  mutate(nmes = factor(str_sub(mes, 6, 7)),
         pais_orig = factor(pais_orig)) |> 
  select(nmes, pais_orig, turistas, gasto) |> 
  filter(str_detect(pais_orig, "Total", negate = TRUE))
  
escenario <- read_csv("ESCENARIO_DESTINO.csv")

# pfutbol <- sum(escenario$Futbol == 1) / nrow(escenario)
# rate_nservicios <- mean(escenario$nservicios)
# rate_capacidad <- mean(escenario$capacidad)
# m_cont <- apply(escenario[,4:25], 2, mean)
# s_cont <- apply(escenario[,4:25], 2, sd)

## de la configuración
nsim <- 100

simulacion <- escenario |> 
  mutate(sim_turistas = list(rpois(nsim, turistas))) |> 
  unnest(sim_turistas) |> 
  select(-turistas)
  
simulacion.x <- simulacion |> 
  mutate(nmes = factor(str_sub(mes, 6, 7), levels = levels(dm$nmes)),
         pais_orig = factor(pais_orig, levels = levels(dm$pais_orig))) |> 
  select(nmes, pais_orig, turistas = sim_turistas) |> 
  model.matrix(~., data = _)

modelo <- read_rds("modelo_xgb.rds")
predict(modelo, simulacion.x)







Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Rows: 24 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): mes, pais_orig, mun_dest
dbl (1): turistas

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `nmes = factor(str_sub(mes, 6, 7), levels =
  levels(dm$nmes))`.
[1mCaused by error in `levels()`:[22m
[33m![39m objeto 'dm' no encontrado
